In [1]:
import sys
sys.path.append('/home/ian/code/github')

In [2]:
from utils.pymongo_wrapper import pymongo_wrapper

In [3]:
pmw = pymongo_wrapper()

In [4]:
zztable = pmw.get_collection('jiankang39','zz')

In [6]:
zzlist = list(pmw.find_all(zztable,fieldlist=['症状名称','相关疾病']))
zzlist[:2]

[{'症状名称': '脑血管痉挛', '相关疾病': ['颅内动脉瘤', '小儿甲状旁腺功能减退症']},
 {'症状名称': '脑干出血',
  '相关疾病': ['高血压性脑出血', '脑出血', '脑胶质瘤', '小脑脑桥角脑膜瘤', '小儿原发性脑干损伤', '小儿脑干胶质瘤']}]

In [7]:
zzs = list({i['症状名称'] for i in zzlist})
len(zzs)

5898

In [11]:
diseases = set()
for i in zzlist:
    diseases = diseases.union(i['相关疾病'])
diseases = list(diseases)
len(diseases)

7068

In [12]:
from utils.pandas_wrapper import pandas_wrapper as pdw 

In [13]:
zzseries = pdw.build_series(zzs)
zzseries[:5]

0       眼睛刺痛
1        足跟痛
2     口腔黏膜脱落
3     皮下脂肪增多
4    甲状腺功能低下
dtype: object

In [15]:
disseries = pdw.build_series(diseases)
disseries[:5]

0      缩窄性心包炎
1    小儿骨与关节结核
2        巨大胎儿
3      药物性食管炎
4       疱疹样湿疹
dtype: object

In [17]:
zzseries_inv = pdw.build_series(zzseries.index,index=zzseries.values)
zzseries_inv[:5]

眼睛刺痛       0
足跟痛        1
口腔黏膜脱落     2
皮下脂肪增多     3
甲状腺功能低下    4
dtype: int64

In [18]:
disseries_inv = pdw.build_series(disseries.index,index=disseries.values)
disseries_inv[:5]

缩窄性心包炎      0
小儿骨与关节结核    1
巨大胎儿        2
药物性食管炎      3
疱疹样湿疹       4
dtype: int64

In [19]:
from utils.numpy_wrapper import numpy_wrapper as npw

In [20]:
zzdismatr = npw.build_zeros_array((len(zzseries),len(disseries)))

In [21]:
for i in zzlist:
    x = zzseries_inv[i['症状名称']]
    for ii in i['相关疾病']:
        zzdismatr[x,disseries_inv[ii]] = 1

In [26]:
diszzmatr = zzdismatr.T

In [75]:
zz1 = ['脑血管痉挛']
temp = npw.build_zeros_array((1,len(zzseries)))[0]
for i in zz1:
    temp[zzseries_inv[i]] = 1

In [78]:
from math import sqrt
def cal_cos_similarity(a1,a2):
    a = a1.dot(a2)
    b = a1.dot(a1)
    c = a2.dot(a2)
    if b == 0 or c==0:
        return 0
    return a/sqrt( b*c )

In [39]:
diszzmatr[1]

array([0., 0., 0., ..., 0., 0., 0.])

In [41]:
diszzmatr.shape

(7068, 5898)

In [42]:
temp.shape

(1, 5898)

In [108]:
def zz2disease(zzs,n=None):
    temp = npw.build_zeros_array((1,len(zzseries)))[0]
    for i in zzs:
        temp[zzseries_inv[i]] = 1
    l1 = list()
    for i in range(len(disseries)):
        l1.append(cal_cos_similarity(temp,diszzmatr[i]))
    s1 = pdw.build_series(l1)
    s2 = s1.sort_values(ascending=False)
    if n:
        list(disseries[s2.iloc[s2.nonzero()].index].values)[:n],list(s2.iloc[s2.nonzero()].values)[:n]
    return list(disseries[s2.iloc[s2.nonzero()].index].values),list(s2.iloc[s2.nonzero()].values)

In [109]:
zz1 = ['脑血管痉挛']
zz2disease(zz1)

(['颅内动脉瘤', '小儿甲状旁腺功能减退症'], [0.35355339059327373, 0.2672612419124244])

In [110]:
zz1 = ['失眠','多梦']
zz2disease(zz1)

(['职业妇女病', '经行失眠', '职业倦怠症', '家庭主妇综征合征', '空巢综合症'],
 [0.7071067811865475,
  0.7071067811865475,
  0.7071067811865475,
  0.7071067811865475,
  0.7071067811865475])

In [111]:
zz1 = ['失眠','多梦','脑水肿','孕妇嗜睡']
zz2disease(zz1)

(['空巢综合症', '家庭主妇综征合征', '职业倦怠症', '职业妇女病', '小儿透析失衡综合征'],
 [0.5, 0.5, 0.5, 0.5, 0.5])

In [114]:
zz1 = ['脑水肿','孕妇嗜睡','神经衰弱综合征','眼睛刺痛']
zz2disease(zz1)

(['孕吐', '小儿透析失衡综合征', '脉络膜病变', '新生儿急性肾功能衰竭', '老年人异位抗利尿素分泌综合征'],
 [0.5, 0.5, 0.35355339059327373, 0.35355339059327373, 0.2886751345948129])

In [91]:
s2.nonzero()

(array([0, 1]),)

In [96]:
list(disseries[s2.iloc[s2.nonzero()].index].values)

['颅内动脉瘤', '小儿甲状旁腺功能减退症']

In [35]:
sqrt(a1.dot(a1))

2.23606797749979